# 单无人机路径规划————最短路径的引导

（注意：运行本程序需要获得task3中生成的env_discretizing.pkl文件）

让我们将<a href="task3_SUAV_RL_Design.ipynb">无人机强化学习任务</a>中单无人机路径规划部分生成的环境文件加载进来

In [1]:
import sys
sys.path.append('..')
import UAVENVmudule_design as UE

import pickle
with open("env_discretizing.pkl", "rb") as file:
    env_discretizing = pickle.load(file)

对于此环境，本节我们用最短路径对无人机进行引导飞行。

我们设计一个**神经网络-蚁群算法**的组合方法，以实现最短路径的引导飞行。在该方法中，我们用神经网络代替蚁群算法中的蚂蚁，用神经网络的输出作为蚂蚁的移动方向，使用强化学习的策略来更新网络参数。

##### 蚂蚁选择路径原理

对于单个蚂蚁，我们将其视为智能体，用神经网络来模拟其行为。神经网络的输入为当前状态，输出为前进的位置。神经网络的训练目标是使其输出与最终路径规划目标越接近越好。  
蚂蚁输入可用三元组$<E,\ D,\ B>$，其中$E$为当前蚂蚁待前进位置周围环境，$D$为待前进位置距离终点的距离，$B$为待前进位置蚂蚁前进过的次数，我们将对这三种信息分别进行相关预处理，使信息特征对齐，再将其输入神经网络，得到最后的待前进位置。

**$E$信息处理**  
$E$表示某位置，或某位置序列的周围环境信息，我们将对周围环境的障碍物信息进行分析，将其映射到0~1，0表示该位置周围环境不好，没有出路；1表示该位置周围环境好，有其他待选的出路。

**$D$信息处理**  
$D$表示待前进位置距离终点的距离，我们将其映射到0~1，0表示该位置距离终点相对较远；1表示该位置距离终点相对较近。

**$B$信息处理**  
$B$表示待前进位置蚂蚁前进过的次数，我们将其映射到0~1，0表示该位置基本没有蚂蚁来过，1表示该位置有很多蚂蚁来过。

我们将三组信息输入一神经网络，得到待前进位置。

##### 训练策略
从某点出发，进行路径选择，直到到达终点。在每一步，我们用神经网络选择待前进位置。最后，利用较好的路径更新网络参数。

In [2]:
env_discretizied = env_discretizing.get_discetized_env()

In [3]:
shape=env_discretizied.shape

In [ ]:
import numpy as np
import torch
def generate_combinations(n:int = 3, element_list:list = [-1,0,1]) -> list:
    '''
    生成一个列表组合，输入n为列表元素个数，element_list为列表元素内容
    '''
    result = []
    if n == 1:
        for i in element_list:
            result.append([i])
        return result
    elif n < 1:
        raise ValueError('n must be greater than 0')
    elif type(n) != int:
        raise TypeError('n must be int')
    else:
        for i in element_list:
            for comb in generate_combinations(n-1, element_list):
                if type(comb) != list:
                    result.append([i] + [comb])
                else:
                    result.append([i] + comb)
        return result

def EDBmessage_get(env_discretizied:np.ndarray, target_position:list, now_position:list):
    '''
    获得当前位置的EDB信息，其中
    env_discretizied为离散化后的空间信息
    target_position为目标位置
    now_position为当前位置
    '''
    if type(env_discretizied) == list:
        env_discretizied = np.array(env_discretizied)
    elif type(env_discretizied) == torch.Tensor:
        env_discretizied = env_discretizied.to('cpu').numpy()
        
    if type(env_discretizied) != np.ndarray:
        raise TypeError('env_discretizied must be np.ndarray')
    
    shape_env = env_discretizied.shape
    size = len(shape_env)
    # 获得待前进位置
    positons_tochoose = generate_combinations(n=3, element_list=[-1,0,1])
    for i in range(len(positons_tochoose)):
        positons_tochoose[i] = [now_position[j] + positons_tochoose[i][j] for j in range(3)]
    while now_position in positons_tochoose:
        positons_tochoose.remove(now_position)
    
    
        
    

In [37]:
positons_tochoose = generate_combinations(n=3, element_list=[-1,0,1])
now_position = [30,30,30]
for i in range(len(positons_tochoose)):
    positons_tochoose[i] = [now_position[j] + positons_tochoose[i][j] for j in range(3)]
while now_position in positons_tochoose:
    positons_tochoose.remove(now_position)
